# ARCHS4 model building with CLAMP

💡 **Environment:** `clamp-analyses`  

This notebook focuses on preprocessing data from the ARCHS4 database. It covers initial cleaning, and preparation for subsequent analyses.

## Load libraries

In [ ]:
library(hdf5r)
library(biomaRt)
library(dplyr)
library(here)
library(CLAMP)

source(here("config.R"))
set.seed(config$ARCHS4$RANDOM_SVD_SEED)

## Output directory

In [45]:
output_dir <- config$ARCHS4$DATASET_FOLDER
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

## Preprocess archs4 data

In [46]:
file_path <- config$ARCHS4$DATASET_FILE

h5        <- H5File$new(file_path, mode = "r")
dset      <- h5[["/data/expression"]]
gene_symbols <- h5[["/meta/genes/symbol"]]$read()
gene_ids     <- h5[["/meta/genes/ensembl_gene"]]$read()
sample_names <- h5[["/meta/samples/geo_accession"]]$read()
sc_samples      <- h5[["/meta/samples/singlecellprobability"]]$read()
lib_strategy <- h5[["/meta/samples/library_strategy"]]$read()     

In [47]:
samples_idx <- which(sc_samples < 0.5)
sample_names_n_singlecell  <- sample_names[samples_idx]

Get genes length

In [48]:
output_file <- file.path(output_dir, "gene_lengths.rds")

ensembl_ver <- config$ARCHS4$DATASET_ENSEMBL_VERSION
ensembl <- useEnsembl(biomart="ensembl", version=ensembl_ver)
mart <- biomaRt::useDataset("hsapiens_gene_ensembl", ensembl)

gene_info <- biomaRt::getBM(
  filters = "hgnc_symbol",
  attributes = c("ensembl_gene_id", "hgnc_symbol", "transcript_length"),
  values = gene_symbols, 
  mart = mart
)

gene_lengths <- gene_info %>%
  group_by(hgnc_symbol, ensembl_gene_id) %>%
  summarize(gene_length_bp = max(transcript_length, na.rm = TRUE)) %>%
  pull(gene_length_bp, name = hgnc_symbol)

`summarise()` has grouped output by 'hgnc_symbol'. You can override using the
`.groups` argument.


In [49]:
# keep in data only genes for which we have lengths
# here gene symbols are duplicated (we'll use that later)
gene_symbols_idx <- which(gene_symbols %in% names(gene_lengths) )
gene_symbols <- gene_symbols[gene_symbols_idx]

# get unique gene symbols
# since we are gonna use rowsum to aggregate same gene symbols, I use it here
# to get a list of unique and fixed gene symbols
summed <- rowsum(t(dset[1:10, gene_symbols_idx]), group = gene_symbols)
gene_symbols_unique <- rownames(summed)
stopifnot(length(gene_symbols_unique) == length(unique(gene_symbols)))
gene_lengths <- gene_lengths[gene_symbols_unique]
n_genes <- length(gene_symbols_unique)
rm(summed)

## FMB matrix

In [50]:
fbm_files <- c(
  file.path(output_dir, "FBMarchs4.bk"),
  file.path(output_dir, "FBMarchs4_preproc.bk"),
  file.path(output_dir, "FBMarchs4_filtered.bk"),
  file.path(output_dir, "FBMarchs4_preproc_filtered.bk"),
  file.path(output_dir, "FBMarchs4_preproc_filtered.rds"),
  file.path(output_dir, "FBMarchs4_nosinglecell.bk"),
  file.path(output_dir, "FBMarchs4_nosinglecell_preproc.bk"),
  file.path(output_dir, "FBMarchs4_nosinglecell_preproc_filtered.bk"),
  file.path(output_dir, "FBMarchs4_nosinglecell_preproc_filtered.rds")
)

for (f in fbm_files) {
  if (file.exists(f)) {
    res <- unlink(f)
    if (identical(res, 0L)) {
      message("✅ Removed existing FBM file: ", f)
    } else {
      warning("⚠️ Failed to remove: ", f)
    }
  } else {
    message("ℹ️ Not found, skipping: ", f)
  }
}

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/archs4/FBMarchs4.bk

ℹ️ Not found, skipping: /home/msubirana/Documents/pivlab/clamp-analyses/output/archs4/FBMarchs4_preproc.bk

ℹ️ Not found, skipping: /home/msubirana/Documents/pivlab/clamp-analyses/output/archs4/FBMarchs4_filtered.bk

ℹ️ Not found, skipping: /home/msubirana/Documents/pivlab/clamp-analyses/output/archs4/FBMarchs4_preproc_filtered.bk

ℹ️ Not found, skipping: /home/msubirana/Documents/pivlab/clamp-analyses/output/archs4/FBMarchs4_preproc_filtered.rds

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/archs4/FBMarchs4_nosinglecell.bk

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/archs4/FBMarchs4_nosinglecell_preproc.bk

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/archs4/FBMarchs4_nosinglecell_preproc_filtered.bk

ℹ️ Not found, skipping: /home/msubirana/Documents/pivlab/clamp-a

In [ ]:
#remove just for testing
n_genes <- length(gene_symbols_unique)
n_samples <- length(sample_names)

In [52]:
# Create the FBM
fbm_file <- file.path(output_dir, "FBMarchs4")
bkfile   <- paste0(fbm_file, ".bk")

archs4FBM <- FBM(
    nrow        = n_genes,
    ncol        = n_samples,
    backingfile = fbm_file,
    create_bk   = TRUE
)

In [53]:
block_size <- 100
n_blocks   <- ceiling(n_samples / block_size)

pb <- txtProgressBar(min = 0, max = n_blocks, style = 3)

for (i in 1:n_blocks) {
  setTxtProgressBar(pb, i)

  start_row <- (i-1) * block_size + 1
  end_row <- min(i * block_size, n_samples)

  raw_block <- NULL
  tryCatch(
    expr = {
      raw_block <- dset[start_row:end_row, ]
    },
    error = function(e) {
      message(paste("Error with block: ", i))
    }
  )
  if (is.null(raw_block)) {
    # FIXME: I add a dummy block here with near-to-zero values; not sure if
    # this is the best approach here; for some reason, there is a block that
    # cannot be read in ARCHS4, so I "skip" it here
    raw_block <- matrix(1e-10, (end_row - start_row + 1), n_genes)
  }

  # subset genes to those with gene length only
  raw_block <- raw_block[, gene_symbols_idx]

  # aggregate duplicated gene symbols
  raw_block_t <- t(raw_block)
  raw_block_summed <- rowsum(raw_block_t, group = gene_symbols)
  raw_block <- t(raw_block_summed)
  rownames(raw_block) <- NULL

  # tpm normalize

  tpm_norm <- function(counts, gene.lengths) {
    if (!is.matrix(counts)) {
      stop("`counts` must be a matrix.")
    }
    counts <- as.matrix(counts)
    
    if (is.null(names(gene.lengths))) {
      stop("`gene.lengths` must be a named numeric vector.")
    }
    if (!all(colnames(counts) %in% names(gene.lengths))) {
      stop("All column names of `counts` must be in names(gene.lengths).")
    }
    
    # Reorder lengths to match columns
    #lengths.bp <- gene.lengths[colnames(counts)]
    
    # Convert lengths to kilobases
    lengths.kb <- gene.lengths / 1e3
    
    # 1) Divide counts by gene length in kilobases → reads per kilobase
    rpk <- sweep(counts, 2, lengths.kb, FUN = "/")
    
    # 2) Compute per-sample scaling factor: sum of RPKs
    per.sample.sum <- rowSums(rpk)
    
    # 3) Divide RPKs by the sum and multiply by 1e6 → TPM
    tpm <- sweep(rpk, 1, per.sample.sum, FUN = "/") * 1e6
    
    return(tpm)
  }

  raw_block <- t(tpm_norm(raw_block, gene_lengths))
  archs4FBM[, start_row:end_row] <- as.matrix(raw_block)
}

  |======================================================================| 100%

Once done, close h5 file

In [54]:
h5$close_all()

## ARCHS4 preprocessing

In [55]:
# remove single cell samples
samples_idx <- which(sample_names %in% sample_names_n_singlecell)
sample_names <- sample_names[samples_idx]
n_samples <- length(sample_names)

archs4FBM <- big_copy(
  archs4FBM,
  ind.col = samples_idx,
  backingfile = file.path(output_dir, "FBMarchs4_nosinglecell")
)

In [56]:
# Preprocess
prep_archs4 <- preprocessCLAMPFBM(
  fbm = archs4FBM,
  mean_cutoff = config$ARCHS4$GENES_MEAN_CUTOFF,
  var_cutoff = config$ARCHS4$GENES_VAR_CUTOFF,
  block_size = block_size
)

archs4_fbm_filt <- prep_archs4$fbm_filtered
archs4_rowStats <- prep_archs4$rowStats

Applying log2 transformation

Filling NAs with 0



In [57]:
archs4_genes <- gene_symbols_unique[prep_archs4$kept_rows]

In [60]:
# Zscore
zscoreCLAMPFBM(
    fbm_filtered=archs4_fbm_filt,
    rowStats=archs4_rowStats,
    chunk_size=block_size)

Applying Z-score transformation



In [61]:
saveRDS(prep_archs4, file = file.path(output_dir, "prep_archs4.rds"))
saveRDS(archs4_genes, file = file.path(output_dir, "archs4_genes.rds"))
saveRDS(sample_names, file = file.path(output_dir, "archs4_samples.rds"))
saveRDS(archs4_fbm_filt, file = file.path(output_dir, "archs4_fbm_filt.rds"))